In [1]:
import argparse
import base64
from datetime import datetime
import os
import shutil
import cv2

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 9
controller.set_desired(set_speed)

cropped_height = 66
cropped_width = 320

# Format image to [66, 320, 1]
def format_image(X):
    X_proc = np.zeros((cropped_height, cropped_width, 1))
    
    for x in range(0, cropped_width-1):
        for y in range(0, cropped_height-1):
            X_proc[y, x, 0] = X[y, x]
    return X_proc


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        ori_image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(ori_image)
        image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
        image = image[70:70 + cropped_height, 0:cropped_width]
        image = format_image(image)
        X_train_data = np.zeros((1, cropped_height, cropped_width, 1))
        # image_array = np.asarray(image)
        # steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))
        X_train_data[0] =  format_image(image)

        steering_angle = float(model.predict(X_train_data, batch_size=1))

        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)
        
        base_path = './recorded_images/'
        # save frame
        timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
        image_filename = os.path.join(base_path, timestamp)
        ori_image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':

    # check that model Keras version is same as local Keras version
    model_file = 'model.h5'
    # f = h5py.File(args.model, mode='r')
    f = h5py.File(model_file, mode='r')    
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    model = load_model(model_file)
    recored_images_folder = 'recorded_images'
    if recored_images_folder != '':
        print("Creating image folder at {}".format(recored_images_folder))
        if not os.path.exists(recored_images_folder):
            os.makedirs(recored_images_folder)
        else:
            shutil.rmtree(recored_images_folder)
            os.makedirs(recored_images_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's mi
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


Using TensorFlow backend.


Creating image folder at recorded_images
RECORDING THIS RUN ...
connect  207558bee1044a6d9d76d68ebfb9401c


(6070) wsgi starting up on http://0.0.0.0:4567
(6070) accepted ('127.0.0.1', 37349)


-0.17357785999774933 0.918
-0.17357785999774933 0.936
-0.17346975207328796 0.9540000000000001
-0.17346975207328796 0.9333522000000001
-0.17346975207328796 0.9505944000000001
-0.17346975207328796 0.9678366
-0.14938685297966003 0.687963
-0.14938685297966003 0.6993794
-0.14938685297966003 0.7107958
-0.13499586284160614 0.4010142
-0.13499586284160614 0.40613259999999995
-0.13499586284160614 0.41125100000000003
-0.12232103198766708 0.18002520000000008
-0.12232103198766708 0.18050940000000007
-0.12232103198766708 0.18099360000000006
-0.08654647320508957 0.05366160000000007
-0.08654647320508957 0.05163960000000008
-0.08654647320508957 0.04961760000000008
0.034418974071741104 -0.017551799999999923
0.034418974071741104 -0.02085119999999993
0.034418974071741104 -0.02415059999999991
0.08268369734287262 0.004557599999999967
0.08268369734287262 0.001885799999999993
0.08268369734287262 -0.0007860000000000089
0.08752036839723587 0.41253900000000004
0.08752036839723587 0.41802400000000006
0.0875203683

-0.020690176635980606 0.07628419999999982
-0.020690176635980606 0.07622699999999982
-0.020690176635980606 0.07616979999999983
-0.014336946420371532 0.07631659999999989
-0.014336946420371532 0.07626339999999988
-0.014336946420371532 0.07621019999999988
-0.03194242715835571 0.07632019999999987
-0.03194242715835571 0.07627019999999987
-0.03194242715835571 0.07622019999999988
-0.045691780745983124 0.07630279999999993
-0.045691780745983124 0.07625539999999993
-0.045691780745983124 0.07620799999999994
-0.02933647111058235 0.07629319999999999
-0.02933647111058235 0.0762484
-0.02933647111058235 0.0762036
-0.026561422273516655 0.07631180000000001
-0.026561422273516655 0.07627000000000002
-0.026561422273516655 0.07622820000000002
-0.013251995667815208 0.07628839999999998
-0.013251995667815208 0.07624859999999997
-0.013251995667815208 0.07620879999999998
-0.02280678227543831 0.07626079999999993
-0.02280678227543831 0.07622279999999994
-0.02280678227543831 0.07618479999999994
-0.009948388673365116

...

-0.020169973373413086 0.07635020000000033
-0.020169973373413086 0.07634980000000033
-0.020169973373413086 0.07634940000000033
-0.03620840981602669 0.07633880000000035
-0.03620840981602669 0.07633820000000036
-0.03620840981602669 0.07633760000000037
-0.024190274998545647 0.07633700000000038
-0.024190274998545647 0.07633640000000037
-0.024190274998545647 0.07633580000000038
-0.028706349432468414 0.07635560000000033
-0.028706349432468414 0.07635540000000032
-0.028706349432468414 0.07635520000000032
-0.027226340025663376 0.07634480000000035
-0.027226340025663376 0.07634440000000035
-0.027226340025663376 0.07634400000000036
-0.03505059331655502 0.07634360000000036
-0.03505059331655502 0.07634320000000036
-0.03505059331655502 0.07634280000000036
-0.0753154382109642 0.07634240000000035
-0.0753154382109642 0.07634200000000035
-0.0753154382109642 0.07634160000000036
-0.04777424409985542 0.07639220000000024
-0.04777424409985542 0.07639280000000025
-0.04777424409985542 0.07639340000000024
-0.0627

-0.024953795596957207 0.07778600000000038
-0.024953795596957207 0.07779480000000039
-0.024953795596957207 0.0778036000000004
-0.02355365641415119 0.07777160000000032
-0.02355365641415119 0.07777960000000031
-0.02355365641415119 0.0777876000000003
-0.03130026534199715 0.07770380000000034
-0.03130026534199715 0.07771000000000033
-0.03130026534199715 0.07771620000000035
-0.0022757695987820625 0.07763060000000038
-0.0022757695987820625 0.07763500000000038
-0.0022757695987820625 0.0776394000000004
0.007071411237120628 0.07758260000000033
0.007071411237120628 0.07758580000000033
0.007071411237120628 0.07758900000000032
0.011145483702421188 0.0775616000000004
0.011145483702421188 0.07756420000000039
0.011145483702421188 0.0775668000000004
0.03904062137007713 0.0775694000000004
0.03904062137007713 0.07757200000000039
0.03904062137007713 0.0775746000000004
0.027702772989869118 0.07760780000000032
0.027702772989869118 0.07761100000000032
0.027702772989869118 0.07761420000000031
0.022002262994647

-0.07304923236370087 0.07594840000000033
-0.07304923236370087 0.07593360000000032
-0.07304923236370087 0.07591880000000031
-0.08215242624282837 0.07659760000000032
-0.08215242624282837 0.07659640000000033
-0.08215242624282837 0.07659520000000034
-0.061856821179389954 0.07683880000000029
-0.061856821179389954 0.0768424000000003
-0.061856821179389954 0.0768460000000003
-0.0945935845375061 0.07675780000000033
-0.0945935845375061 0.07675960000000034
-0.0945935845375061 0.07676140000000034
-0.11699047684669495 0.07652860000000034
-0.11699047684669495 0.07652580000000034
-0.11699047684669495 0.07652300000000033
-0.10121111571788788 0.07655080000000043
-0.10121111571788788 0.07654860000000044
-0.10121111571788788 0.07654640000000043
-0.11675913631916046 0.07656460000000039
-0.11675913631916046 0.07656280000000038
-0.11675913631916046 0.07656100000000038
-0.10503304749727249 0.0764368000000003
-0.10503304749727249 0.0764326000000003
-0.10503304749727249 0.0764284000000003
-0.10154260694980621 

-0.0020987067837268114 1.8075214000000004
-0.0015374114736914635 1.8253670000000004
-0.0015374114736914635 1.8433426000000002
0.022061794996261597 1.8614100000000002
0.016765758395195007 1.8799994000000002
0.016765758395195007 1.8979888000000003
0.004881339613348246 1.9152744000000004
0.007283995859324932 1.9333724000000005
0.007283995859324932 1.9513504000000004
0.017778657376766205 1.9685940000000004
0.019310319796204567 1.9871186000000005
0.019310319796204567 2.0050932000000006
0.011873843148350716 2.0230576000000005
0.012499802745878696 2.0410932000000006
0.012499802745878696 2.0590688000000004
0.024652210995554924 2.0766670000000005
0.0238568764179945 2.0939620000000003
0.0238568764179945 2.1119170000000005
0.03450767695903778 2.1307492000000003
0.02870759554207325 2.1488846000000006
0.02870759554207325 2.1668600000000007
0.011166821233928204 2.1848354000000008
0.013254983350634575 2.202861800000001
0.013254983350634575 2.2208382000000007


127.0.0.1 - - [22/Jun/2017 09:10:54] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 159.165214
wsgi exiting
(6070) wsgi exited, is_accepting=True
